In [149]:
import csv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [329]:
circle_data = []
trig_data = []

def clean_data(data):
    ndata = []
    data_loss = 0
    for e in data:
        if e == "":
            data_loss +=1
            continue
        ndata.append(int(float(e)))
    for _ in range(data_loss):
        ndata.append(0)
    return ndata

# 打开CSV文件
with open('0.csv', mode='r', encoding='utf-8') as file:
    # 创建CSV阅读器
    reader = csv.reader(file)
    # 逐行读取CSV文件
    for row in reader:
        row = row[:-1]
        row = clean_data(row)
        circle_data.append(row)

# 打开CSV文件
with open('1.csv', mode='r', encoding='utf-8') as file:
    # 创建CSV阅读器
    reader = csv.reader(file)
    # 逐行读取CSV文件
    for row in reader:
        row = row[:-1]
        row = clean_data(row)
        trig_data.append(row)

t_c_x = circle_data[:-20]
v_c_x = circle_data[-20:]
t_t_x = trig_data[:-20]
v_t_x = trig_data[-20:]

In [369]:
c_d = tf.Variable(initial_value=t_c_x, dtype=tf.float32)
t_d = tf.Variable(initial_value=t_t_x, dtype=tf.float32)
train_data = tf.concat([c_d,t_d], axis = 0)

labble1 = tf.zeros(shape=(101, 1), dtype=tf.float32)
labble2 = tf.ones(shape=(115, 1), dtype=tf.float32)
labble = tf.concat([labble1,labble2], axis = 0)

# 计算最大值和最小值
data_min = tf.reduce_min(train_data)
data_max = tf.reduce_max(train_data)

# 应用最小-最大缩放公式
scaled_data = 2 * ((train_data - data_min) / (data_max - data_min)) - 1

# 检查结果
print(c_d.shape) # (101, 120)
print(t_d.shape) # (115, 120)
print(scaled_data)

(101, 120)
(115, 120)
tf.Tensor(
[[-3.7986636e-03 -3.7986636e-03 -3.7986636e-03 ... -3.7986636e-03
  -3.7986636e-03 -3.7986636e-03]
 [ 2.3285234e-01 -2.3239285e-01 -8.8518214e-01 ... -5.9387922e-01
   8.7185621e-02 -7.0067090e-01]
 [ 2.2770584e-01 -1.5318394e-04 -9.9555802e-01 ... -4.7973531e-01
   7.2450399e-02 -6.1342400e-01]
 ...
 [ 6.0349822e-02 -3.5444021e-02 -5.6367397e-02 ...  3.2987165e-01
  -2.5242805e-02  1.6193366e-01]
 [-3.7986636e-03 -3.7986636e-03 -3.7986636e-03 ... -3.7986636e-03
  -3.7986636e-03 -3.7986636e-03]
 [-3.7986636e-03 -3.7986636e-03 -3.7986636e-03 ... -3.7986636e-03
  -3.7986636e-03 -3.7986636e-03]], shape=(216, 120), dtype=float32)


In [400]:
W1 = tf.Variable(initial_value = tf.random.uniform(shape = (120, 50)))
b1 = tf.Variable(initial_value = tf.zeros(shape=(50,)))
W2 = tf.Variable(initial_value = tf.random.uniform(shape = (50, 18)))
b2 = tf.Variable(initial_value = tf.zeros(shape=(18,)))
W3 = tf.Variable(initial_value = tf.random.uniform(shape = (18, 1)))
b3 = tf.Variable(initial_value = tf.zeros(shape=(1,)))

In [401]:
# 定义模型
def model(inputs):
    l1 = tf.matmul(inputs, W1) + b1
    l2 = tf.matmul(tf.nn.relu(l1), W2) + b2
    l3 = tf.matmul(tf.nn.relu(l2), W3) + b3
    return tf.sigmoid(l3)

In [402]:
# 定义损失函数 为 方差
def square_loss(targets, predictions):
    per_sample_losses = tf.square(targets - predictions)
    return  tf.reduce_mean(per_sample_losses)

In [403]:
la = 0.3

def training_step(inputs, targets):
    with tf.GradientTape() as tape:
        y = model(inputs)
        loss = square_loss(targets, y)
    W1l,b1l,W2l,b2l,W3l,b3l = tape.gradient(loss, [W1,b1,W2,b2,W3,b3])
    W1.assign_sub(W1l*la)
    W2.assign_sub(W2l*la)
    W3.assign_sub(W3l*la)
    b1.assign_sub(b1l*la)
    b2.assign_sub(b2l*la)
    b3.assign_sub(b3l*la)
    return loss


In [404]:
for step in range(1000):
    loss = training_step(train_data, labble)
    print(f"Loss at step {step}: {loss:.4f}")

Loss at step 0: 0.4988
Loss at step 1: 0.4982
Loss at step 2: 0.4975
Loss at step 3: 0.4969
Loss at step 4: 0.4962
Loss at step 5: 0.4956
Loss at step 6: 0.4950
Loss at step 7: 0.4944
Loss at step 8: 0.4938
Loss at step 9: 0.4932
Loss at step 10: 0.4927
Loss at step 11: 0.4921
Loss at step 12: 0.4916
Loss at step 13: 0.4911
Loss at step 14: 0.4906
Loss at step 15: 0.4900
Loss at step 16: 0.4895
Loss at step 17: 0.4891
Loss at step 18: 0.4886
Loss at step 19: 0.4881
Loss at step 20: 0.4877
Loss at step 21: 0.4872
Loss at step 22: 0.4868
Loss at step 23: 0.4864
Loss at step 24: 0.4860
Loss at step 25: 0.4855
Loss at step 26: 0.4851
Loss at step 27: 0.4848
Loss at step 28: 0.4844
Loss at step 29: 0.4840
Loss at step 30: 0.4836
Loss at step 31: 0.4833
Loss at step 32: 0.4829
Loss at step 33: 0.4826
Loss at step 34: 0.4822
Loss at step 35: 0.4819
Loss at step 36: 0.4816
Loss at step 37: 0.4813
Loss at step 38: 0.4810
Loss at step 39: 0.4807
Loss at step 40: 0.4804
Loss at step 41: 0.4801
Lo

In [406]:
vc = tf.Variable(initial_value=v_c_x, dtype=tf.float32)
vt = tf.Variable(initial_value=v_t_x, dtype=tf.float32)
model(vc)

<tf.Tensor: shape=(20, 1), dtype=float32, numpy=
array([[1.        ],
       [0.86252075],
       [1.        ],
       [0.86252075],
       [1.        ],
       [0.86252075],
       [1.        ],
       [0.86252075],
       [1.        ],
       [1.        ],
       [1.        ],
       [0.86252075],
       [1.        ],
       [0.86252075],
       [1.        ],
       [1.        ],
       [1.        ],
       [0.86252075],
       [1.        ],
       [1.        ]], dtype=float32)>